<a href="https://colab.research.google.com/github/saba-ramezani/C4I_System2/blob/master/C4I_System_chatbot_persian_nervals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 36.1 MB/s 
     |████████████████████████████████| 120 kB 70.6 MB/s 
     |████████████████████████████████| 6.6 MB 56.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 18.8 MB/s 


In [ ]:
PROJECT_DATA='/content/drive/MyDrive/C4I_system_chatbot/C4I_System/data'

In [ ]:
import json
with open(PROJECT_DATA + "/ner_intent_persion_nervals.json") as f:
  ner_intent = json.load(f)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
NER_ITEMS = sorted(set([t2 for t1 in ner_intent for t2 in ner_intent[t1]['NERTAGS']]),reverse=True)  
print(len(NER_ITEMS), NER_ITEMS)

INTENT_ITEMS = sorted(set([t2 for t1 in ner_intent for t2 in ner_intent[t1]['INTENTS']]),reverse=True)  
print(len(INTENT_ITEMS),INTENT_ITEMS)

14 ['O', 'I-staff_name', 'I-area', 'B-staff_name', 'B-sensor_type_2', 'B-sensor_type_1', 'B-sensor_status_1', 'B-sensor_name_2', 'B-sensor_name_1', 'B-rank', 'B-parameter', 'B-barracks_name_2', 'B-barracks_name_1', 'B-area']
22 ['is_staff_active', 'is_barracks_insider', 'get_type_of_two_barracks_link', 'get_type_of_barracks', 'get_sub_barracks_of_barracks', 'get_status_of_two_barracks_link', 'get_sensors_status_based_on_location_and_sensor_type', 'get_sensor_type', 'get_sensor_count_based_on_sensor_type', 'get_sensor_count_based_on_sensor_status', 'get_sensor_IP', 'get_parameter_of_sensor_based_on_parameter_type', 'get_enemy_sensors_based_on_barracks_id', 'get_count_of_barracks_staff_based_on_rank', 'get_coordinates_of_sensor', 'get_coordinates_of_barracks', 'get_channel_of_two_barracks_link', 'get_barracks_name_of_a_staff', 'get_all_sensors_that_do_not_interfere_based_on_sensor_type', 'get_all_parameters_of_sensor', 'get_access_level_of_a_staff', 'check_if_two_sensors_interfere']


In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer

model_name = "HooshvareLab/bert-fa-zwnj-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/292 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [ ]:
def encode_texts(tokenizer, texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors="tf")

texts = [d for d in ner_intent]
tds = encode_texts(tokenizer, texts)
tds.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
encoded_texts = tds

In [ ]:
intent_map = dict() # index -> intent
for idx, ui in enumerate(INTENT_ITEMS):
    intent_map[ui] = idx
intent_map

{'is_staff_active': 0,
 'is_barracks_insider': 1,
 'get_type_of_two_barracks_link': 2,
 'get_type_of_barracks': 3,
 'get_sub_barracks_of_barracks': 4,
 'get_status_of_two_barracks_link': 5,
 'get_sensors_status_based_on_location_and_sensor_type': 6,
 'get_sensor_type': 7,
 'get_sensor_count_based_on_sensor_type': 8,
 'get_sensor_count_based_on_sensor_status': 9,
 'get_sensor_IP': 10,
 'get_parameter_of_sensor_based_on_parameter_type': 11,
 'get_enemy_sensors_based_on_barracks_id': 12,
 'get_count_of_barracks_staff_based_on_rank': 13,
 'get_coordinates_of_sensor': 14,
 'get_coordinates_of_barracks': 15,
 'get_channel_of_two_barracks_link': 16,
 'get_barracks_name_of_a_staff': 17,
 'get_all_sensors_that_do_not_interfere_based_on_sensor_type': 18,
 'get_all_parameters_of_sensor': 19,
 'get_access_level_of_a_staff': 20,
 'check_if_two_sensors_interfere': 21}

In [ ]:
import numpy as np
# # map to train_data values
def encode_intents(intents, intent_map):
    encoded = np.zeros((len(intents),len(intent_map)))
    for idx,intnt in enumerate(intents):
        encoded[ idx, [intent_map[i] for i in intnt]] = 1
    # convert to tf tensor

    return tf.convert_to_tensor(encoded, dtype="int32")

encoded_intents = encode_intents([ ner_intent[t]['INTENTS'] for t in ner_intent], intent_map)
encoded_intents

<tf.Tensor: shape=(16095, 22), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [ ]:
np.argmax(encoded_intents[0]),ner_intent[next(iter(ner_intent))]

(8,
 {'TEXT': ['چند',
   'سنسور',
   'از',
   'نوع',
   'رادار',
   'مربوط',
   'به',
   'پادگان',
   '1',
   'وجود',
   'دارد؟'],
  'NERTAGS': ['O',
   'O',
   'O',
   'O',
   'B-sensor_type_1',
   'O',
   'O',
   'O',
   'B-barracks_name_1',
   'O',
   'O'],
  'NERVALS': {'sensor_type_1': 'رادار', 'barracks_name_1': '1'},
  'INTENTS': ['get_sensor_count_based_on_sensor_type']})

In [ ]:
slot_map = dict() # slot -> index
for idx, us in enumerate(NER_ITEMS):
    slot_map[us] = idx
slot_map

{'O': 0,
 'I-staff_name': 1,
 'I-area': 2,
 'B-staff_name': 3,
 'B-sensor_type_2': 4,
 'B-sensor_type_1': 5,
 'B-sensor_status_1': 6,
 'B-sensor_name_2': 7,
 'B-sensor_name_1': 8,
 'B-rank': 9,
 'B-parameter': 10,
 'B-barracks_name_2': 11,
 'B-barracks_name_1': 12,
 'B-area': 13}

In [ ]:
import numpy as np

# find the max encoded test length
# tokenizer pads all texts to same length anyway so
# just get the length of the first one's input_ids
#max_len = len(encoded_texts["input_ids"][0])
max_len = max([len(x) for x in encoded_texts["input_ids"]])
#print(max_len)
def encode_slots(all_slots, all_texts, 
                 toknizer, slot_map, max_len=max_len):
    encoded_slots = np.zeros(shape=(len(all_texts), max_len), dtype=np.int32)
    
    for idx, text in enumerate(all_texts):
        enc = [] # for this idx, to be added at the end to encoded_slots
        
        # slot names for this idx
        slot_names = all_slots[idx]
        print(text, slot_names)
        # raw word tokens
        # not using bert for this block because bert uses
        # a wordpiece tokenizer which will make 
        # the slot label to word mapping
        # difficult
        raw_tokens = text.split()

        # words or slot_values associated with a certain
        # slot_name are contained in the values of the
        # dict slots_names
        # now this becomes a two way lookup
        # first we check if a word belongs to any
        # slot label or not and then we add the value from
        # slot map to encoded for that word
        for rt,rt_slot_name in zip(raw_tokens,slot_names):
            # use bert tokenizer
            # to get wordpiece tokens
            bert_tokens = tokenizer.tokenize(rt)
            
            # find the slot name for a token
            #rt_slot_name = get_slot_from_word(rt, slot_names)
            #rt_slot_name = get_slot_from_word(rt, slot_names) Moved to for and ziped
            if rt_slot_name is not None:
                # fill with the slot_map value for all ber tokens for rt
                enc.append(slot_map[rt_slot_name])
                enc.extend([slot_map[rt_slot_name]] * (len(bert_tokens) - 1))

            else:
                # rt is not associated with any slot name
                enc.append(0)
                
        
        # now add to encoded_slots
        # ignore the first and the last elements
        # in encoded text as they're special chars
        encoded_slots[idx, 1:len(enc)+1] = enc
    
    return encoded_slots

In [ ]:
# all_slots = [td.slots for td in train_data]
# all_texts = [td.text for td in train_data]


all_slots = [ s['NERTAGS'] for t,s in ner_intent.items() ]
all_texts = [ t for t in ner_intent ]

In [ ]:
encoded_slots = encode_slots(all_slots, all_texts, tokenizer, slot_map)
print(encoded_slots)
encoded_slots[2],all_slots[2],all_texts[2]
encoded_slots.shape

# **Classifier Model**

In [ ]:
from transformers import TFBertModel
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy

class JointIntentAndSlotFillingModel(tf.keras.Model):

    def __init__(self, intent_num_labels=None, slot_num_labels=None,
                 model_name=model_name, dropout_prob=0.1):
        super().__init__(name="joint_intent_slot")
        self.bert = TFBertModel.from_pretrained(model_name)
        self.dropout = Dropout(dropout_prob)
        # self.intent_classifier = Dense(intent_num_labels,
        #                                name="intent_classifier")

        self.intent_classifier = Dense(intent_num_labels, activation='sigmoid',
                                       name="intent_classifier")

        self.slot_classifier = Dense(slot_num_labels,
                                     name="slot_classifier")

    def call(self, inputs, **kwargs):
        # two outputs from BERT
        trained_bert = self.bert(inputs, **kwargs)
        pooled_output = trained_bert.pooler_output
        sequence_output = trained_bert.last_hidden_state
        
        # sequence_output will be used for slot_filling / classification
        sequence_output = self.dropout(sequence_output,
                                       training=kwargs.get("training", False))
        slot_logits = self.slot_classifier(sequence_output)

        # pooled_output for intent classification
        pooled_output = self.dropout(pooled_output,
                                     training=kwargs.get("training", False))
        intent_logits = self.intent_classifier(pooled_output)
        return slot_logits, intent_logits
        #return {'slot': slot_logits,'intent': intent_logits} 

In [ ]:
joint_model = JointIntentAndSlotFillingModel(
    intent_num_labels=len(intent_map), slot_num_labels=len(slot_map))

In [ ]:
#https://towardsdatascience.com/multi-label-image-classification-in-tensorflow-2-0-7d4cf8a4bc72
@tf.function
def macro_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.
    
    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        
    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

@tf.function
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [ ]:
encoded_slots.shape[1], encoded_intents.shape[1]

In [ ]:
categorcial_slots = tf.keras.utils.to_categorical(encoded_slots, num_classes=len(slot_map))
categorcial_slots.shape, categorcial_slots.shape[-1]

In [ ]:
class MacroF1(tf.keras.metrics.Metric):

  def __init__(self, name='MacroF1', **kwargs):
    super(MacroF1, self).__init__(name=name, **kwargs)
    self.true_positives = self.add_weight(name='tp', initializer='zeros')

  def update_state(self, y_true, y_pred, sample_weight=None):
    thresh = 0.5
    y_hat =  tf.cast(tf.reshape(y_pred, [-1,11]),tf.float32)
   
    y2 = tf.cast(y_true, tf.float32)
    y2 = tf.keras.utils.to_categorical(y2, num_classes=11)
    y2 = tf.reshape(y2,[-1])
    #oh_labels = tf.cast(tf.one_hot(tf.cast(labels, tf.int32), num_classes), dtype=labels.dtype)
    #print(y)
    #y2 = tf.cast(tf.one_hot(tf.cast(y2, tf.int32), 11), dtype=tf.int32)
    y =  tf.cast(y2 , tf.float32)

    print(y.shape, y_hat)
    print(y.shape, y_hat)
    print(y.shape, y_hat)
    print(y.shape, y_hat)
    print(y.shape, y_hat)
    print(y.shape, y_hat)

    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)

  def result(self):
    return self.true_positives

In [ ]:
import tensorflow_addons as tfa
tf.config.run_functions_eagerly(True)
class MY_F1Score(tfa.metrics.FBetaScore):
    def __init__(
        self,
        num_classes: 11,
        average = None,
        threshold = None,
        name = "MY_F1Score",
        dtype = None,
    ):
        super().__init__(num_classes, average, 1.0, threshold, name=name, dtype=dtype)


    def get_config(self):
        base_config = super().get_config()
        del base_config["beta"]
        return base_config

    def update_state(self, y_true, y_pred, sample_weight=None):   
        #print(y_true,y_pred) 
        #y_true_ = tf.keras.utils.to_categorical(y_true, num_classes=11)
        y_true_ = tf.constant(tf.reshape(y_true,[-1,y_true.shape[-1]])   )
        y_pred_ = tf.constant(tf.reshape(y_pred,[-1,y_pred.shape[-1]])   )
        #print(y_true_,y_pred_) 
 
        super().update_state(y_true_, y_pred_, sample_weight=None)

In [ ]:
opt = Adam(learning_rate=3e-5, epsilon=1e-08)

# two outputs, one for slots, another for intents
# we have to fine tune for both
# losses = [SparseCategoricalCrossentropy(from_logits=True),
#           'binary_crossentropy'
#           #macro_soft_f1
#           ]

losses = [CategoricalCrossentropy(from_logits=True),
          'binary_crossentropy'
          #macro_soft_f1
          ]
                    
# losses = {'slot': CategoricalCrossentropy(from_logits=True),'intent': 'binary_crossentropy'}

#from tensorflow_addons import metrics
#metric1 = tfa.metrics.F1Score(num_classes=encoded_slots.shape[1], average = 'macro', threshold=0.7, name='m1')
metric2 = tfa.metrics.F1Score(num_classes=encoded_intents.shape[1], average = 'macro', threshold=0.7, name='m2')


#metrics = ['accuracy']
#metrics = [ metric1, metric2]
# metric1 = MY_F1Score(num_classes=11, average = 'micro', threshold=0.7, name='m1')
# metrics = {'slot':metric1, "'intent'":metric2} 
metrics = [ MY_F1Score(num_classes=11, average = 'micro', threshold=0.7, name='m1')]

# compile model
joint_model.compile(optimizer=opt, loss=losses, loss_weights = [1,1], metrics=metrics)

In [ ]:
x = {"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}

#history = joint_model.fit(x, (encoded_slots, encoded_intents), epochs=1, batch_size=32, shuffle=True)
history = joint_model.fit(x, (categorcial_slots, encoded_intents), epochs=3, batch_size=32, shuffle=True)
#history = joint_model.fit(x, {'slot': categorcial_slots,'intent': encoded_intents}, epochs=1, batch_size=32, shuffle=True)

In [ ]:
res = joint_model.predict(x)
res

In [ ]:
# test = MY_F1Score(num_classes=11, average = 'macro', threshold=0.7, name='m1')

# test.update_state(encoded_slots,res[0])
# test.result()

In [ ]:
import numpy as np
joint_model.save_weights(PROJECT_DATA + "/checkpoints/my_checkpoint")
a = np.random.random((10))
print(a,np.nonzero(a>0.5)[0])

In [ ]:
def map2cat(t):
    return t.replace("B-", "").replace("I-", "") 

def nlu(text, tokenizer, model, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = model(inputs)
    #print("output:",outputs)
    slot_logits, intent_logits = outputs

    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, :]
    #print("slot_ids:",slot_ids)
    #intent_id = intent_logits.numpy().argmax(axis=-1)[0]
    intent_ids = np.nonzero(intent_logits.numpy()>0.8)[1]
    #print(intent_logits,np.nonzero(intent_logits.numpy()>0.5))
    #print("intent_ids:",intent_ids)
    #info = {"intent": intent_names[intent_id], "slots": {}}
    info = {"intent": [intent_names[intent_id] for intent_id in intent_ids], "slots": {}}
    #print("info:", info)
    out_dict = {}
    # get all slot names and add to out_dict as keys
    predicted_slots = set([slot_names[s] for s in slot_ids if s != 0])
    #print("predicted_slots:",predicted_slots)
    #merge slots
    predicted_slots = set([ map2cat(t) for t in predicted_slots])
    #print("predicted_slots:",predicted_slots)
    #print(predicted_slots)
    for ps in predicted_slots:
      out_dict[ps] = []

    # check if the text starts with a small letter
    #if text[0].islower():
    #  tokens = tokenizer.tokenize(text, add_special_tokens=True)
    #else:
    #add speial tokens for persian 
    tokens = tokenizer.tokenize(text, add_special_tokens=True)
    #print("tokens:", tokens)
    for token, slot_id in zip(tokens, slot_ids):
        # add all to out_dict
        #slot_name = slot_names[slot_id]
        slot_name = map2cat(slot_names[slot_id])

        if slot_name == "O":
            continue


        # collect tokens
        collected_tokens = [token]
        idx = tokens.index(token)

        
        # see if it starts with ##
        # then it belongs to the previous token
        if token.startswith("##"):
          # check if the token already exists or not
          if tokens[idx - 1] not in out_dict[slot_name]:
           collected_tokens.insert(0, tokens[idx - 1])


        # add collected tokens to slots
        out_dict[slot_name].extend(collected_tokens)

    #print("out dict:", out_dict)
    # process out_dict
    for slot_name in out_dict:
        tokens = out_dict[slot_name]
        slot_value = tokenizer.convert_tokens_to_string(tokens)

        info["slots"][slot_name] = slot_value.strip()

    return info

t_itrator=iter(ner_intent)
for i in range(3):
    tkey = next(t_itrator)
    print(tkey, nlu(tkey, tokenizer, joint_model, INTENT_ITEMS, NER_ITEMS), ner_intent[tkey]['INTENTS'], ner_intent[tkey]['NERVALS'])
    print()

In [ ]:
nlu("انتنهای شمال شرق کشور چه وضعیتی دارند؟", tokenizer, joint_model, 
    INTENT_ITEMS, NER_ITEMS)